1. **Input Parameters:**
   - $p_{\text{int}}$: Initial transmit powers matrix of shape $N \times K$.
   - $\mathbf{H}$: Channel gains matrix of shape $N \times K \times K$.
   - $P_{\text{max}}$: Maximum transmit power constraint.
   - $\text{var\_noise}$: Variance of the noise.

2. **Initialization:**
   - $N = \text{p\_int.shape}[0]$
   - $K = \text{p\_int.shape}[1]$
   - Initialize $\mathbf{b} = \sqrt{\text{p\_int}}$
   - Initialize $\mathbf{f}$ and $\mathbf{w}$ matrices of shape $N \times K \times 1$ with zeros.

3. **Calculation of Received Power:**
   - $\text{rx\_power} = \mathbf{H} \cdot \mathbf{b}$
   - $\text{valid\_rx\_power} = \sum_{k=1}^{K} (\text{rx\_power}[n][k][k])$ for $n = 1$ to $N$
   - $\text{interference} = \sum_{k=1}^{K} (\text{rx\_power}[n][k][k]^2) + \text{var\_noise}$

4. **Initialization of $\mathbf{f}$ and $\mathbf{w}$:**
   - $\mathbf{f}[n] = \frac{\text{valid\_rx\_power}[n]}{\text{interference}[n]}$ for $n = 1$ to $N$
   - $\mathbf{w}[n] = \frac{1}{1 - \mathbf{f}[n] \cdot \text{valid\_rx\_power}[n]}$ for $n = 1$ to $N$

5. **Iterations:**
   - For $\text{ii} = 1$ to 100:
     - Update $\text{rx\_power}$ using $\mathbf{b}$ and $\mathbf{f}$.
     - Update $\text{bup} = \mathbf{w} \cdot \text{valid\_rx\_power}$.
     - Update $\text{bdown} = \sum_{k=1}^{K} (\text{rx\_power}[n][k][k]^2 \cdot \mathbf{w})$ for $n = 1$ to $N$.
     - Update $\mathbf{b} = \min(\text{btmp}, \mathbf{1} \cdot \sqrt{P_{\text{max}}}) + \max(\text{btmp}, \mathbf{0}) - \text{btmp}$.
     - Update $\text{interference}$ using updated $\mathbf{b}$ and $\mathbf{f}$.
     - Update $\mathbf{f}$ and $\mathbf{w}$ similarly as in initialization.

6. **Optimal Transmit Powers:**
   - $\text{p\_opt} = \text{np.square}(\mathbf{b})$

7. **Output:**
   - $\text{p\_opt}$: Optimal transmit powers matrix of shape $N \times K$.


In [ ]:
def batch_WMMSE(p_int, H, Pmax, var_noise):
    N = p_int.shape[0]
    K = p_int.shape[1]
    vnew = 0
    b = np.sqrt(p_int)
    f = np.zeros((N,K,1) )
    w = np.zeros( (N,K,1) )
    

    mask = np.eye(K)
    rx_power = np.multiply(H, b)
    rx_power_s = np.square(rx_power)
    valid_rx_power = np.sum(np.multiply(rx_power, mask), 1)
    
    interference = np.sum(rx_power_s, 2) + var_noise
    f = np.divide(valid_rx_power,interference)
    w = 1/(1-np.multiply(f,valid_rx_power))
    #vnew = np.sum(np.log2(w),1)
    
    
    for ii in range(100):
        fp = np.expand_dims(f,1)
        rx_power = np.multiply(H.transpose(0,2,1), fp)
        valid_rx_power = np.sum(np.multiply(rx_power, mask), 1)
        bup = np.multiply(w,valid_rx_power)
        rx_power_s = np.square(rx_power)
        wp = np.expand_dims(w,1)
        bdown = np.sum(np.multiply(rx_power_s,wp),2)
        btmp = bup/bdown
        b = np.minimum(btmp, np.ones((N,K) )*np.sqrt(Pmax)) + np.maximum(btmp, np.zeros((N,K) )) - btmp
        
        bp = np.expand_dims(b,1)
        rx_power = np.multiply(H, bp)
        rx_power_s = np.square(rx_power)
        valid_rx_power = np.sum(np.multiply(rx_power, mask), 1)
        interference = np.sum(rx_power_s, 2) + var_noise
        f = np.divide(valid_rx_power,interference)
        w = 1/(1-np.multiply(f,valid_rx_power))
    p_opt = np.square(b)
    return p_opt

1. **Input Parameters:**
   - $p_{\text{int}}$: Initial transmit powers matrix of shape $N \times K$.
   - $\alpha$: User-specific scaling factors matrix of shape $N \times K$.
   - $\mathbf{H}$: Channel gains matrix of shape $N \times K \times K$.
   - $P_{\text{max}}$: Maximum transmit power constraint.
   - $\text{var\_noise}$: Variance of the noise.

2. **Initialization:**
   - $N = \text{p\_int.shape}[0]$
   - $K = \text{p\_int.shape}[1]$
   - Initialize $\mathbf{b} = \sqrt{\text{p\_int}}$
   - Initialize $\mathbf{f}$ and $\mathbf{w}$ matrices of shape $N \times K \times 1$ with zeros.

3. **Calculation of Received Power:**
   - $\text{rx\_power} = \mathbf{H} \cdot \mathbf{b}$
   - $\text{rx\_power\_s} = \text{np.square}(\text{rx\_power})$
   - $\text{valid\_rx\_power} = \sum_{k=1}^{K} (\text{rx\_power}[n][k][k])$ for $n = 1$ to $N$
   - $\text{interference} = \sum_{k=1}^{K} (\text{rx\_power\_s}[n][k][k]) + \text{var\_noise}$

4. **Initialization of $\mathbf{f}$ and $\mathbf{w}$:**
   - $\mathbf{f}[n] = \frac{\text{valid\_rx\_power}[n]}{\text{interference}[n]}$ for $n = 1$ to $N$
   - $\mathbf{w}[n] = \frac{1}{1 - \mathbf{f}[n] \cdot \text{valid\_rx\_power}[n]}$ for $n = 1$ to $N$

5. **Iterations:**
   - For $\text{ii} = 1$ to 100:
     - Update $\text{rx\_power}$ using $\mathbf{b}$ and $\mathbf{f}$.
     - Update $\text{bup} = \alpha \cdot \mathbf{w} \cdot \text{valid\_rx\_power}$.
     - Update $\text{bdown} = \sum_{k=1}^{K} (\alpha \cdot \text{rx\_power\_s}[n][k][k] \cdot \mathbf{w})$ for $n = 1$ to $N$.
     - Update $\mathbf{b}$ using $\text{bup}$ and $\text{bdown}$.
     - Update $\text{interference}$ using updated $\mathbf{b}$ and $\mathbf{f}$.
     - Update $\mathbf{f}$ and $\mathbf{w}$ similarly as in initialization.

6. **Optimal Transmit Powers:**
   - $\text{p\_opt} = \text{np.square}(\mathbf{b})$

7. **Output:**
   - $\text{p\_opt}$: Optimal transmit powers matrix of shape $N \times K$.


In [ ]:
def batch_WMMSE2(p_int, alpha, H, Pmax, var_noise):
    N = p_int.shape[0]
    K = p_int.shape[1]
    vnew = 0
    b = np.sqrt(p_int)
    f = np.zeros((N,K,1) )
    w = np.zeros( (N,K,1) )
    

    mask = np.eye(K)
    rx_power = np.multiply(H, b)
    rx_power_s = np.square(rx_power)
    valid_rx_power = np.sum(np.multiply(rx_power, mask), 1)
    
    interference = np.sum(rx_power_s, 2) + var_noise
    f = np.divide(valid_rx_power,interference)
    w = 1/(1-np.multiply(f,valid_rx_power))
    #vnew = np.sum(np.log2(w),1)
    
    
    for ii in range(100):
        fp = np.expand_dims(f,1)
        rx_power = np.multiply(H.transpose(0,2,1), fp)
        valid_rx_power = np.sum(np.multiply(rx_power, mask), 1)
        bup = np.multiply(alpha,np.multiply(w,valid_rx_power))
        rx_power_s = np.square(rx_power)
        wp = np.expand_dims(w,1)
        alphap = np.expand_dims(alpha,1)
        bdown = np.sum(np.multiply(alphap,np.multiply(rx_power_s,wp)),2)
        btmp = bup/bdown
        b = np.minimum(btmp, np.ones((N,K) )*np.sqrt(Pmax)) + np.maximum(btmp, np.zeros((N,K) )) - btmp
        
        bp = np.expand_dims(b,1)
        rx_power = np.multiply(H, bp)
        rx_power_s = np.square(rx_power)
        valid_rx_power = np.sum(np.multiply(rx_power, mask), 1)
        interference = np.sum(rx_power_s, 2) + var_noise
        f = np.divide(valid_rx_power,interference)
        w = 1/(1-np.multiply(f,valid_rx_power))
    p_opt = np.square(b)
    return p_opt

# Function: WMMSE_sum_rate

This function computes the optimal power allocation that maximizes the sum rate using the Weighted Minimum Mean Squared Error (WMMSE) algorithm.

### Inputs:
- $p_{\text{int}}$: Initial power allocation vector.
- $H$: Channel gains matrix of size $K \times K$.
- $P_{\text{max}}$: Maximum transmit power.
- $\text{var}_{\text{noise}}$: Variance of the noise.

### Mathematical Formulation:
1. **Objective Function:**
   - The objective is to maximize the sum rate, which can be formulated as:
     $$
     \text{maximize} \quad \sum_{i=1}^{K} \log_2(w_i)
     $$
   where $w_i$ is the weighted rate for user $i$.

2. **Initialization:**
   - Initialize the power allocation vector $b$ as the square root of the initial power allocation vector $p_{\text{int}}$.

3. **Scaling Factors:**
   - Compute the scaling factors $f_i$ and $w_i$ for each user $i$:
     $$
     f_i = \frac{H_{ii} \cdot b_i}{\sum_{j=1}^{K} H_{ij}^2 \cdot b_j^2 + \text{var}_{\text{noise}}}
     $$
     $$
     w_i = \frac{1}{1 - f_i \cdot H_{ii} \cdot b_i}
     $$
   where $H_{ii}$ is the channel gain for user $i$.

4. **Iterative Optimization:**
   - Update the power allocation vector $b$ iteratively until convergence:
     $$
     b_i = \min\left( \frac{w_i \cdot f_i \cdot H_{ii}}{\sum_{j=1}^{K} w_j \cdot f_j^2 \cdot H_{ji}^2}, \sqrt{P_{\text{max}}} \right) + \max\left( \frac{w_i \cdot f_i \cdot H_{ii}}{\sum_{j=1}^{K} w_j \cdot f_j^2 \cdot H_{ji}^2}, 0 \right) - \frac{w_i \cdot f_i \cdot H_{ii}}{\sum_{j=1}^{K} w_j \cdot f_j^2 \cdot H_{ji}^2}
     $$
   - Convergence is achieved when the change in the objective function value is below a threshold.

5. **Optimal Power Allocation:**
   - The optimal power allocation $p_{\text{opt}}$ is obtained by squaring the final power allocation vector $b$:
     $$
     p_{\text{opt}} = b^2
     $$

### Outputs:
- $p_{\text{opt}}$: Optimal power allocation vector maximizing the sum rate.


In [ ]:
def WMMSE_sum_rate(p_int, H, Pmax, var_noise):
    K = np.size(p_int)
    vnew = 0
    b = np.sqrt(p_int)
    f = np.zeros(K)
    w = np.zeros(K)
    for i in range(K):
        f[i] = H[i, i] * b[i] / (np.square(H[i, :]) @ np.square(b) + var_noise)
        w[i] = 1 / (1 - f[i] * b[i] * H[i, i])
        vnew = vnew + math.log2(w[i])

    VV = np.zeros(100)
    for iter in range(100):
        vold = vnew
        for i in range(K):
            btmp = w[i] * f[i] * H[i, i] / sum(w * np.square(f) * np.square(H[:, i]))
            b[i] = min(btmp, np.sqrt(Pmax)) + max(btmp, 0) - btmp

        vnew = 0
        for i in range(K):
            f[i] = H[i, i] * b[i] / ((np.square(H[i, :])) @ (np.square(b)) + var_noise)
            w[i] = 1 / (1 - f[i] * b[i] * H[i, i])
            vnew = vnew + math.log2(w[i])

        VV[iter] = vnew
        if vnew - vold <= 1e-3:
            break

    p_opt = np.square(b)
    return p_opt

# Function: get_mu

This function computes the value of $\mu$ used in the power allocation algorithm.

### Inputs:
- $P_{\text{max}}$: Maximum transmit power.
- $H_{kk}$: Channel gains for a specific user $k$.
- $b_{\text{tmp}}$: Temporary power allocation.

### Mathematical Formulation:
1. **Initialization:**
   - $R_{\mu} = \frac{\sqrt{P_{\text{max}}}}{\|H_{kk}\|}$
   - $R_{\mu}$ is the upper bound for $\mu$, defined as the ratio of the square root of the maximum transmit power to the norm of the channel gains for user $k$.

   - $L_{\mu} = 0$
   - $N$: Number of elements in the channel gains matrix $H_{kk}$.
   - $P_{\text{comp}} = H_{kk}^T (b_{\text{tmp}}^{-2}) H_{kk}$

2. **Binary Search:**
   - While $R_{\mu} - L_{\mu} > 1e-1$:
     - $mid_{\mu} = \frac{R_{\mu} + L_{\mu}}{2}$
     - $P_{\text{comp}} = H_{kk}^T (b_{\text{tmp}} + mid_{\mu} I)^{-2} H_{kk}$
     - If $P_{\text{comp}} < P_{\text{max}}$:
       - $R_{\mu} = mid_{\mu}$
     - Else:
       - $L_{\mu} = mid_{\mu}$

3. **Output:**
   - $\mu$: Value used in the power allocation algorithm.


In [ ]:
def get_mu(Pmax,Hkk,btmp):
    Rmu = np.sqrt(Pmax)/np.norm(Hkk)
    Lmu = 0
    N = Hkk.shape[0]
    Pcomp = np.matmul(np.matmul(Hkk.T,np.linalg.matrix_power(np.inv(btmp),2)),Hkk)
    if(Pcomp < Pmax):
        return Lmu

    I = eye(N)
    while(Rmu-Lmu > 1e-1):
    	midmu = (Rmu + Lmu)/2
    	Pcomp = np.matmul(np.matmul(Hkk.T,np.linalg.matrix_power(np.inv(btmp + midmu*I),2)),Hkk)
    	if(Pcomp < Pmax):
    		Rmu = midmu
    	else: 
    		Lmu = midmu
    return Lmu

# Function: WMMSE_vector

This function computes the optimal power allocation vector using the Weighted Minimum Mean Square Error (WMMSE) algorithm.

### Inputs:
- $ b_{\text{int}} $: Initial power allocation vector.
- $ \alpha $: Weighting coefficients.
- $ H $: Channel matrix of dimensions $ K \times K \times N $.
- $ P_{\text{max}} $: Maximum transmit power.
- $ \text{var}_{\text{noise}} $: Variance of the noise.

### Mathematical Formulation:
1. **Initialization:**
   - Initialize $ K $, $ N $, and $ v_{\text{new}} $.
   - Set $ b = b_{\text{int}} $.
   - Initialize arrays $ f $ and $ w $ with zeros.
   - Compute the initial value of $ f_k $ for each user $ k $:
     $$ f_k = \frac{\sum_{n=1}^{N} H_{kk}^{(n)} b_k^{(n)}}{\text{var}_{\text{noise}} + \sum_{j=1}^{K} \sum_{n=1}^{N} |H_{kj}^{(n)}|^2 b_j^{(n)}} $$
   - Compute the initial value of $ w_k $ for each user $ k $:
     $$ w_k = \frac{1}{1 - w_k f_k} $$
   - $ w_k $ represents the weight factor associated with user $ k $ in the power allocation process. It reflects the impact of user $ k $'s allocated power on the overall sum rate
   - Compute the initial value of $ v_{\text{new}} $:
     $$ v_{\text{new}} = \sum_{k=1}^{K} \log_2(w_k) $$
   - $ v_{\text{new}} $ is the total sum rate achieved by the current power allocation vector

2. **Iterative Optimization:**
   - Perform iterative optimization to find the optimal power allocation vector.
   - For each iteration:
     - Update the power allocation vector $ b_k $ for each user $ k $:
       $$ b_k = \left(\sum_{j=1}^{K} w_j |f_j|^2 H_{kj} H_{kj}^T + \mu_k I \right)^{-1} H_{kk} \cdot w_k \cdot f_k $$
       where $ \mu_k $ is the Lagrange multiplier obtained using a binary search to satisfy the power constraint.
     - Update the value of $ v_{\text{new}} $:
       $$ v_{\text{new}} = \sum_{k=1}^{K} \log_2(w_k) $$
     - Check convergence criteria (difference between $ v_{\text{new}} $ and $ v_{\text{old}} $).
     - If convergence criteria met, exit the loop.

3. **Output:**
   - $ b $: Optimal power allocation vector.


In [ ]:
def WMMSE_vector(b_int, alpha, H, Pmax, var_noise):
    '''H: K * K * N
    f: K * 1
    w: K * 1
    b: K * N 
    '''
    K = b_int.shape[0]
    N = b_int.shape[1]
    vnew = 0
    b = b_int
    f = np.zeros(K)
    w = np.zeros(K)
    for ii in range(K):
    	ftmp = var_noise
    	f[ii] = np.dot(H[ii,ii,:],b[ii,:])
    	for jj in range(K):
    		ftmp += np.square(np.dot(H[ii,jj,:],b[jj,:]))
    	f[ii] = f[ii] / ftmp
    	w[i] = 1 / (1 - w[ii]*f[ii])
    	vnew = vnew + math.log2(w[i])

    VV = np.zeros(100)
    for iter in range(100):
        vold = vnew
        for ii in range(K):
        	btmp = np.zeros((N,N))
       		for jj in range(K):
       			Hjk = np.expand_dims(H[jj,ii,:],axis=0)
       			btmp = btmp + w[jj]*np.square(f[jj])*np.matmul(Hjk.T,Hjk)
       		Hkk = H[ii,ii,:]
       		I = eye(N)
       		mu = get_mu(Pmax/np.square(w[ii]*f[ii]),Hkk,btmp)
       		b[ii] = np.matmul(np.inv(btmp + mu*eye(N)),Hkk) * w[ii] * f[ii]


        vnew = 0
        for ii in range(K):
            ftmp = var_noise
            Hkk = H[ii,ii,:]
            f[ii] = np.dot(Hkk,b[ii,:])
            w[i] = 1 / (1 - b[i]*f[ii])
            for jj in range(K):
                Hkj = H[ii,jj,:]
                ftmp += np.square(np.dot(Hkj,b[jj,:]))
            f[ii] = f[ii] / ftmp
            vnew = vnew + math.log2(w[i])

        VV[iter] = vnew
        if vnew - vold <= 1e-3:
            break

    return b

The function `WMMSE_sum_rate2` computes the optimal power allocation vector using the Weighted Minimum Mean Square Error (WMMSE) algorithm with weighted coefficients.

### Inputs:
- $ p_{\text{int}} $: Initial power allocation vector.
- $ \alpha $: Weighting coefficients for each user.
- $ H $: Channel matrix of dimensions $ K \times K $.
- $ P_{\text{max}} $: Maximum transmit power.
- $ \text{var}_{\text{noise}} $: Variance of the noise.

### Mathematical Formulation:
1. **Initialization:**
   - Initialize $ K $ and $ v_{\text{new}} $.
   - Set $ b = \sqrt{p_{\text{int}}} $.
   - Initialize arrays $ f $ and $ w $ with zeros.
   - Compute the initial value of $ f_k $ for each user $ k $:
     $$ f_k = \frac{H_{kk} \cdot b_k}{\sum_{j=1}^{K} (H_{kj} \cdot b_j)^2 + \text{var}_{\text{noise}}} $$
   - Compute the initial value of $ w_k $ for each user $ k $:
     $$ w_k = \frac{1}{1 - f_k \cdot b_k \cdot H_{kk}} $$
   - Compute the initial value of $ v_{\text{new}} $:
     $$ v_{\text{new}} = \sum_{k=1}^{K} \log_2(w_k) $$

2. **Iterative Optimization:**
   - Perform iterative optimization to find the optimal power allocation vector.
   - For each iteration:
     - Update the power allocation vector $ b_k $ for each user $ k $:
       $$ b_k = \min(\max(\alpha_k \cdot w_k \cdot f_k \cdot H_{kk} \cdot (\sum_{j=1}^{K} \alpha_j \cdot w_j \cdot f_j \cdot H_{kj})^{-1}, \sqrt{P_{\text{max}}}), 0) $$
     - Update the value of $ v_{\text{new}} $:
       $$ v_{\text{new}} = \sum_{k=1}^{K} \log_2(w_k) $$
     - Check convergence criteria (difference between $ v_{\text{new}} $ and $ v_{\text{old}} $).
     - If convergence criteria met, exit the loop.

3. **Output:**
   - $ p_{\text{opt}} $: Optimal power allocation vector.


In [ ]:
def WMMSE_sum_rate2(p_int, alpha, H, Pmax, var_noise):
    K = np.size(p_int)
    vnew = 0
    b = np.sqrt(p_int)
    f = np.zeros(K)
    w = np.zeros(K)
    for i in range(K):
        f[i] = H[i, i] * b[i] / (np.square(H[i, :]) @ np.square(b) + var_noise)
        w[i] = 1 / (1 - f[i] * b[i] * H[i, i])
        vnew = vnew + math.log2(w[i])

    VV = np.zeros(100)
    for iter in range(100):
        vold = vnew
        for i in range(K):
            btmp = alpha[i] * w[i] * f[i] * H[i, i] / sum(alpha * w * np.square(f) * np.square(H[:, i]))
            b[i] = min(btmp, np.sqrt(Pmax)) + max(btmp, 0) - btmp

        vnew = 0
        for i in range(K):
            f[i] = H[i, i] * b[i] / ((np.square(H[i, :])) @ (np.square(b)) + var_noise)
            w[i] = 1 / (1 - f[i] * b[i] * H[i, i])
            vnew = vnew + math.log2(w[i])

        VV[iter] = vnew
        if vnew - vold <= 1e-3:
            break

    p_opt = np.square(b)
    return p_opt

The `perf_eval` function evaluates the performance of different power allocation strategies based on their achieved sum-rate.

### Inputs:
- $ H $: Channel matrix of dimensions, $ K \times K \times \text{num\_sample} $.
- $ \text{Py\_p} $: Power allocation matrix obtained by the Py algorithm, with dimensions, $ K \times \text{num\_sample} $.
- $ \text{NN\_p} $: Power allocation matrix obtained by the neural network model, with dimensions. $ \text{num\_sample} \times K $.
- $ K $: Number of users.
- $ \text{var\_noise} $: Variance of the noise.

### Mathematical Formulation:
1. **Performance Evaluation:**
   - Initialize arrays for Py rate, NN rate, Max power rate, and Random power rate.
   - For each sample:
     - Compute the sum-rate achieved by the Py algorithm: $ \text{pyrate}[i] = \text{obj\_IA\_sum\_rate}(H[:, :, i], \text{Py\_p}[:, i], \text{var\_noise}, K) $.
     - Compute the sum-rate achieved by the neural network model: $ \text{nnrate}[i] = \text{obj\_IA\_sum\_rate}(H[:, :, i], \text{NN\_p}[i, :], \text{var\_noise}, K) $.
     - Compute the sum-rate achieved by allocating equal power to all users: $ \text{mprate}[i] = \text{obj\_IA\_sum\_rate}(H[:, :, i], \mathbf{1}, \text{var\_noise}, K) $.
     - Compute the sum-rate achieved by random power allocation: $ \text{rdrate}[i] = \text{obj\_IA\_sum\_rate}(H[:, :, i], \text{random}(K,1), \text{var\_noise}, K) $.

2. **Print Results:**
   - Print the average sum-rate for each power allocation strategy and the ratio of DNN rate, Max Power rate, and Random Power rate compared to the WMMSE rate.

3. **Plot Histogram:**
   - Plot a histogram showing the distribution of sum-rates achieved by the WMMSE algorithm and the DNN model.
   - Save the histogram plot as an EPS file named 'Histogram_$K.eps'.

### Output:
- The function returns 0 after printing results and plotting the histogram.


In [ ]:
# Functions for performance evaluation
def perf_eval(H, Py_p, NN_p, K, var_noise=1):
    num_sample = H.shape[2]
    pyrate = np.zeros(num_sample)
    nnrate = np.zeros(num_sample)
    mprate = np.zeros(num_sample)
    rdrate = np.zeros(num_sample)
    for i in range(num_sample):
        pyrate[i] = obj_IA_sum_rate(H[:, :, i], Py_p[:, i], var_noise, K)
        nnrate[i] = obj_IA_sum_rate(H[:, :, i], NN_p[i, :], var_noise, K)
        mprate[i] = obj_IA_sum_rate(H[:, :, i], np.ones(K), var_noise, K)
        rdrate[i] = obj_IA_sum_rate(H[:, :, i], np.random.rand(K,1), var_noise, K)
    print('Sum-rate: WMMSE: %0.3f, DNN: %0.3f, Max Power: %0.3f, Random Power: %0.3f'%(sum(pyrate)/num_sample, sum(nnrate)/num_sample, sum(mprate)/num_sample, sum(rdrate)/num_sample))
    print('Ratio: DNN: %0.3f%%, Max Power: %0.3f%%, Random Power: %0.3f%%\n' % (sum(nnrate) / sum(pyrate)* 100, sum(mprate) / sum(pyrate) * 100, sum(rdrate) / sum(pyrate) * 100))

    plt.figure('%d'%K)
    plt.style.use('seaborn-deep')
    data = np.vstack([pyrate, nnrate]).T
    bins = np.linspace(0, max(pyrate), 50)
    plt.hist(data, bins, alpha=0.7, label=['WMMSE', 'DNN'])
    plt.legend(loc='upper right')
    plt.xlim([0, 8])
    plt.xlabel('sum-rate')
    plt.ylabel('number of samples')
    plt.savefig('Histogram_%d.eps'%K, format='eps', dpi=1000)
    plt.show()
    return 0

### Inputs:
- $ K $: Number of users.
- $ \text{num\_H} $: Number of samples.
- $ P_{\text{max}} $: Maximum transmit power.
- $ P_{\text{min}} $: Minimum transmit power.
- $ \text{seed} $: Seed value for random number generation.

### Mathematical Formulation:
1. **Generate Data:**
   - $ P_{\text{ini}} = P_{\text{max}} \times \mathbf{1}_K $
   - $ \text{var\_noise} = 1 $
   - Initialize arrays $ X $ and $ Y $ to store channel matrix and corresponding sum-rates.
   - For each sample $ \text{loop} $ from 1 to $ \text{num\_H} $:
     - Generate a complex Gaussian channel $ \text{CH} $ with elements drawn from $ \mathcal{N}(0, 1) + j \cdot \mathcal{N}(0, 1) $.
     - Compute the absolute value of $ \text{CH} $ to obtain the channel matrix $ H $.
     - Flatten $ H $ to form a vector $ X[:, \text{loop}] $.
     - Compute the sum-rate $ Y[:, \text{loop}] $ using the WMMSE algorithm with initial power $ P_{\text{ini}} $, channel matrix $ H $, maximum power $ P_{\text{max}} $, and noise variance $ \text{var\_noise} $.
     - Update $ \text{total\_time} $ with the time taken for the WMMSE computation.

2. **Return:**
   - Return the channel matrix $ X $, sum-rates $ Y $, and the total computation time.


In [ ]:
def generate_Gaussian(K, num_H, Pmax=1, Pmin=0, seed=2017):
    print('Generate Data ... (seed = %d)' % seed)
    np.random.seed(seed)
    Pini = Pmax*np.ones(K)
    var_noise = 1
    X=np.zeros((K**2,num_H))
    Y=np.zeros((K,num_H))
    total_time = 0.0
    for loop in range(num_H):
        CH = 1/np.sqrt(2)*(np.random.randn(K,K)+1j*np.random.randn(K,K))
        H=abs(CH)
        X[:,loop] = np.reshape(H, (K**2,), order="F")
        H=np.reshape(X[:,loop], (K,K), order="F")
        mid_time = time.time()
        Y[:,loop] = WMMSE_sum_rate(Pini, H, Pmax, var_noise)
        total_time = total_time + time.time() - mid_time
    # print("wmmse time: %0.2f s" % total_time)
    return X, Y, total_time

# Functions for Data Generation: Gaussian IC Half User Case

### Inputs:
- $ K $: Number of users.
- $ \text{num\_H} $: Number of samples.
- $ P_{\text{max}} $: Maximum transmit power.
- $ P_{\text{min}} $: Minimum transmit power.
- $ \text{seed} $: Seed value for random number generation.

### Mathematical Formulation:
1. **Generate Testing Data:**
   - $ P_{\text{ini}} = P_{\text{max}} \times \mathbf{1}_K $
   - $ \text{var\_noise} = 1 $
   - Initialize arrays $ X $ and $ Y $ to store channel matrix and corresponding sum-rates.
   - For each sample $ \text{loop} $ from 1 to $ \text{num\_H} $:
     - Generate a complex Gaussian channel $ \text{CH} $ with elements drawn from $ \mathcal{N}(0, 1) + j \cdot \mathcal{N}(0, 1) $.
     - Compute the absolute value of $ \text{CH} $ to obtain the channel matrix $ H $.
     - Compute the sum-rate for the first $ K $ users using the WMMSE algorithm with initial power $ P_{\text{ini}} $, channel matrix $ H $, maximum power $ P_{\text{max}} $, and noise variance $ \text{var\_noise} $.
     - Update $ \text{total\_time} $ with the time taken for the WMMSE computation.
     - Create an output channel matrix $ OH $ with size $ 2K \times 2K $ and fill the upper-left quadrant with the values of $ H $.
     - Flatten $ OH $ to form a vector $ X[:, \text{loop}] $.

2. **Return:**
   - Return the channel matrix $ X $, sum-rates $ Y $, and the total computation time.


In [ ]:
# Functions for data generation, IMAC case
def generate_IMAC(num_BS, num_User, num_H, Pmax=1, var_noise = 1):
    # Load Channel Data
    CH = sio.loadmat('IMAC_%d_%d_%d' % (num_BS, num_User, num_H))['X']
    Temp = np.reshape(CH, (num_BS, num_User * num_BS, num_H), order="F")
    H = np.zeros((num_User * num_BS, num_User * num_BS, num_H))
    for iter in range(num_BS):
        H[iter * num_User:(iter + 1) * num_User, :, :] = Temp[iter, :, :]

    # Compute WMMSE output
    Y = np.zeros((num_User * num_BS, num_H))
    Pini = Pmax * np.ones(num_User * num_BS)
    start_time = time.time()
    for loop in range(num_H):
        Y[:, loop] = WMMSE_sum_rate(Pini, H[:, :, loop], Pmax, var_noise)
    wmmsetime=(time.time() - start_time)
    # print("wmmse time: %0.2f s" % wmmsetime)
    return CH, Y, wmmsetime, H